In [1]:
#  Installing the required libraries
!pip install selenium beautifulsoup4 pandas


In [19]:
#Importing the required libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# === Initial Setup Code ===
city = input("Enter city name (e.g., mumbai, delhi, pune, bangalore): ").strip().lower()
url = f"https://housing.com/in/buy/{city}"

print(f"\n🔍 Scraping properties for: {city.title()}")

# Setting up the  chromedriver path
CHROMEDRIVER_PATH = r"C:\Users\NILESH VIDHANI\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument("--start-maximized")

service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)

# === Loading the Page ===
driver.get(url)

try:
    # Wait for card containers
    WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.XPATH, "//article[@data-testid='card-container']"))
    )
    time.sleep(2)

    cards = driver.find_elements(By.XPATH, "//article[@data-testid='card-container']")
    print(f"🏘 Found {len(cards)} property cards")

    data = []

    for card in cards:
        # 🔹 Title
        try:
            title = card.find_element(By.XPATH, ".//h2").text.strip()
        except:
            title = "N/A"

        # 🔹 Price
        try:
            price = card.find_element(By.XPATH, ".//div[contains(text(), '₹')]").text.strip()
        except:
            price = "N/A"

        # 🔹 Link
        try:
            link = card.find_element(By.TAG_NAME, "a").get_attribute("href")
        except:
            link = "N/A"

        # 🔹 Location (from subtitle-style class)
        try:
            subtitle = card.find_element(By.CLASS_NAME, "subtitle-style").text.strip()
            if " in " in subtitle:
                location = subtitle.split(" in ")[-1]
            else:
                location = "N/A"
        except:
            location = "N/A"

        data.append({
            "Title": title,
            "Price": price,
            "Location": location,
            "Link": link
        })

    # === Step 3: Save ===
    driver.quit()
    df = pd.DataFrame(data)
    filename = f"housing_{city}.csv"
    df.to_csv(filename, index=False, encoding='utf-8-sig')

    print(f"\n Scraping completed.\n Data saved to: {filename}")

except Exception as e:
    print(" Error during scraping:", str(e))
    driver.quit()


Enter city name (e.g., mumbai, delhi, pune, bangalore):  pune



🔍 Scraping properties for: Pune
🏘️ Found 30 property cards

✅ Scraping completed.
📄 Data saved to: housing_pune.csv
